# Implementación el método de la máxima verosimilitud para la regresión logística

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### PRIMER PASO: Definir la función de entorno L(b)
(máxima verosimilitud)

In [59]:
from IPython.display import display, Math, Latex
display(Math(r"L(\beta) = \sum_{i=0}^n P_i^{y_i} (1-P_i)^{1-y_i}"))

<IPython.core.display.Math object>

In [38]:
def likelihood(y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1, pi[i], 1-pi[i]) #donde si y=1, entonces nos quedamos con pi[i] y en caso contratio 1-pi[i]
        total_sum = total_sum * sum_in[i]
    return total_sum

### SEGUNDO PASO: Calcular las probabilidades para cada observación

In [10]:
display(Math(r'P_i = P(x_i) = \frac{1}{1+e^{-\sum_{j=0}^k\beta_j\cdot x_{ij}}} '))

<IPython.core.display.Math object>

In [32]:
def logitprobs(X,beta):
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi=list(range(1,n_rows+1))
    expon=list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex=X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"):
            pi[i]=1/(1+np.exp(-expon[i]))
    return pi

### TERCER PASO: Calcular la matriz diagonal W

In [11]:
display(Math(r"W=diag(P_i \cdot (1-P_i))_{i=1}^n"))

<IPython.core.display.Math object>

In [20]:
def findW(pi):
    import numpy as np
    n=len(pi)
    W=np.zeros(n*n).reshape(n,n) # creo una matriz de ceros de nxn
    for i in range(n):
        print(i)
        W[i,i]=pi[i]*(1-pi[i])
        W[i,i].astype(float)
    return W

### CUARTO PASO: Obtener la solución de la función logística invocando el método de Newton Raphson

In [22]:
display(Math(r"\beta_{n+1} = \beta_n -\frac{f(\beta_n)}{f'(\beta_n)}"))
print("")
display(Math(r"f(\beta) = X(Y-P)"))
print("")
display(Math(r"f'(\beta) = XWX^T"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [45]:
def logistics(X, Y, limit): 
    import numpy as np
    from numpy import linalg # linalg para hacer la matriz inversa
    nrow = np.shape(X)[0]
    bias = np.ones(nrow).reshape(nrow,1) # hacemos una matriz de unos y la ponemos en fila con el reshape
    X_new = np.append(X, bias, axis = 1)
    ncol = np.shape(X_new)[1]
    beta = np.zeros(ncol).reshape(ncol,1) # hacemos una matriz de ceros y con el reshape la ponemos en columna
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1) 
    iter_i = 10000
    iteracion=1 ###
    while(iter_i>limit):
        print("")
        print(f"Iteración número: {iteracion}") ###
        print("Iter:i"+str(iter_i) + ", limit:" + str(limit))
        pi = logitprobs(X_new, beta)
        print("Pi:"+str(pi))
        W = findW(pi)
        print("W:"+str(W))
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose()) # np.matrix transforma en matriz
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(den)*num)
        beta = beta + root_dif
        print("Beta: "+str(beta))
        iter_i = np.sum(root_dif*root_dif)
        ll = likelihood(Y, pi)
        iteracion +=1 ###
    return beta

## PASO FINAL: Comprobación experimental

In [24]:
import numpy as np

In [25]:
X = np.array(range(10)).reshape(10,1)

In [26]:
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [29]:
Y = [0,0,0,0,1,0,1,0,1,1]

In [30]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X, bias, axis=1)
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [46]:
a=logistics(X,Y,0.00001)


Iteración número: 1
Iter:i10000, limit:1e-05
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta: [[ 0.43636364]
 [-2.36363636]]

Iteración número: 2
Iter:i5.777190082644626, limit:1e-05
Pi:[array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), array([0.56329497]), array([0.666

In [ ]:
# de lo de arriba, sacamos que 0.66220827 es el coeficiente de la primera variable (X) y -3.69557172 es la ordenada en el origen

In [47]:
ll = likelihood(Y, logitprobs(X,a))

In [48]:
ll

array([1.32622426e-06])

In [49]:
# el modelo sería:
Y = 0.66220827 * X -3.69557172

In [ ]:
# qué pasaría si lo contrastamos con el paquete de python:


# Con el paquete statsmodel de python

In [56]:
import statsmodels.api as sm
import pandas as pd
from pandas import Timestamp

In [73]:
Y = [0,0,0,0,1,0,1,0,1,1]

In [74]:
#Y = (Y - np.min(Y))/np.ptp(Y) (si lo quito o lo pongo da lo mismo)
logit_model = sm.Logit(Y,X_new)

In [75]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [76]:
print(result.summary2())

                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.360   
Dependent Variable: y                AIC:              12.6202 
Date:               2023-03-28 12:39 BIC:              13.2254 
No. Observations:   10               Log-Likelihood:   -4.3101 
Df Model:           1                LL-Null:          -6.7301 
Df Residuals:       8                LLR p-value:      0.027807
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
-----------------------------------------------------------------
x1        0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const    -3.6956     2.2889   -1.6145   0.1064   -8.1818   0.7906

